## Nuclei segmentation

#### Preparations

To test different segmentation algorithms, we need access to a test image. When you execute the cell below, a github repository will be cloned and test images will be available in your session. 

You can see available files by clicking the folder icon in the left side menu. 

Alternatively you can upload files directly from your drive to the session sotrage by using the upload button (in the Files tab).

In [ ]:
!git clone https://github.com/fjorka/Pitt_SBII_2023.git

The cell below installs and imports necessary libraries which make available the functions to open, visualize and segment images.

In [ ]:
!pip install cellpose
!pip install scikit-image

In [ ]:
import pandas as pd
import numpy as np
from tifffile import imread, imsave
import matplotlib.pyplot as plt
from cellpose import models
from skimage.measure import regionprops_table

#### Open and display test image

In [ ]:
# define pathway to the image
im_path = './Pitt_SBII_2023/2187T-5_2.0.4_P0007_Z00_DAPI.tif'

# load in image to a variable 'im'
im = imread(im_path)

In [ ]:
# display the test image
plt.imshow(im)

#### Segmentation

Before testing different segmentation algorithms, let's create a small sample image.

In [ ]:
# cut the sample image
im_small = im[1000:1200,1000:1200]

# display the test image
plt.imshow(im_small)

Cellpose library provides following pre-trained models:

'cyto', 'nuclei', 'tissuenet', 'livecell', 'cyto2', 'CP', 'CPx', 'TN1', 'TN2', 'TN3', 'LC1', 'LC2', 'LC3', 'LC4'

Let's test 'nuclei' model first:

In [ ]:
# define the model (if used the first time in the session the model will be imported)
model = models.CellposeModel(gpu=False, model_type='nuclei')

# perform segmentation and store the result in the 'mask' image
mask,_,_ = model.eval(im_small)

In [ ]:
# visualize the results

fig,axes = plt.subplots(1,2,figsize=(15,10))

# display the original sample image
axes[0].imshow(im_small)

# display segmentation mask (each objects is displayed in different color)
axes[1].imshow(mask,cmap='inferno',interpolation='nearest')

Now, let's try a 'TN1' model. 

Please note that this time we define the expected size of the objects by specifying the 'diameter' parameter. You can specify it for any of the cellpose models. 

Also, in this quick test we are not changing the names of the variables. After executing the cell above we will change both the model and the resulting mask - remember that the order of executing cells matter.

In [ ]:
# define the model (if it is used for the first time in the session the model will be imported)
model = models.CellposeModel(gpu=False, model_type='TN1')

# perform segmentation and store the result in the 'mask' image
mask,_,_ = model.eval(im_small,diameter = 10)

In [ ]:
# visualize the results

fig,axes = plt.subplots(1,2,figsize=(15,10))

# display the original sample image
axes[0].imshow(im_small)

# display segmentation mask (each objects is displayed in different color)
axes[1].imshow(mask,cmap='inferno',interpolation='nearest')

Test a few other models to find the best segmentation.

In [ ]:
# define the model (if it is used for the first time in the session the model will be imported)
model = models.CellposeModel(gpu=False, model_type= )

# perform segmentation and store the result in the 'mask' image
mask,_,_ = model.eval(im_small,diameter = 10)

In [ ]:
# visualize the results

fig,axes = plt.subplots(1,2,figsize=(15,10))

# display the original sample image
axes[0].imshow(im_small)

# display segmentation mask (each objects is displayed in different color)
axes[1].imshow(mask,cmap='inferno',interpolation='nearest')

Once you identified the best segmentation algorithm, let's segment the whole image (it should run around 3 min):

In [ ]:
# define the model (if it is used for the first time in the session the model will be imported)
model = models.CellposeModel(gpu=True, model_type= 'cyto')

# perform segmentation and store the result in the 'mask' image
mask,_,_ = model.eval(im,diameter = 25)

In [ ]:
# visualize the results

fig,axes = plt.subplots(1,2,figsize=(15,10))

# display the original sample image
axes[0].imshow(im)

# display segmentation mask (each objects is displayed in different color)
axes[1].imshow(mask,cmap='inferno',interpolation='nearest')

#### Calculate properties of cells

Once a mask is defined, calculating signals of individual objects is a straightforward task.

In [ ]:
# read in and visualize different channels

im_cy3_path = im_path = './Pitt_SBII_2023/2187T-5_2.0.4_P0007_Z00_Cy3.tif'
im_cy3 = imread(im_cy3_path)

im_cy5_path = im_path = './Pitt_SBII_2023/2187T-5_2.0.4_P0007_Z00_Cy5.tif'
im_cy5 = imread(im_cy5_path)

im_fitc_path = im_path = './Pitt_SBII_2023/2187T-5_2.0.4_P0007_Z00_FITC.tif'
im_fitc = imread(im_fitc_path)

In [ ]:
# visualize all the channels

fig,axes = plt.subplots(1,4,figsize=(15,10))

# display the original sample image
axes[0].imshow(im)
axes[0].set_title('DAPI')

axes[1].imshow(im_fitc,vmax=10000)
axes[1].set_title('FITC')

axes[2].imshow(im_cy3,vmax=10000)
axes[2].set_title('Cy3')

axes[3].imshow(im_cy5,vmax=4000)
axes[3].set_title('Cy5')

# remove ticks from the display
for ax in axes:

    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
# define which properties of cells to calculate 
# see documentation here: https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops

selected_properties = ['label','centroid','area','mean_intensity']

# put all the intensity channels together to create a nice package
all_channels = np.stack([im,im_fitc,im_cy3,im_cy5],axis=2)

# calculate properties of cells
my_data = regionprops_table(mask, intensity_image = all_channels, properties = selected_properties)

# put results in a nice table
my_data = pd.DataFrame(my_data)

In [ ]:
my_data